In [1]:
%pip install --upgrade -q \
    keras-nlp==0.12.1 \
    keras==3.3.3 \
    \
    kagglehub==0.2.5


Note: you may need to restart the kernel to use updated packages.


## Download model

In [2]:
import os

# TODO: Create Kaggle API token from https://www.kaggle.com/settings
os.environ["KAGGLE_USERNAME"] = "[TODO]"
os.environ["KAGGLE_KEY"] = "[TODO]"

import kagglehub

# Download latest version
model_path = kagglehub.model_download("keras/llama3/keras/llama3_instruct_8b_en")

print("Path to model files:", model_path)

Path to model files: /home/work/.cache/kagglehub/models/keras/llama3/keras/llama3_instruct_8b_en/3


## Download dataset

In [3]:
!mkdir -p datasets
!wget \
    https://huggingface.co/datasets/databricks/databricks-dolly-15k/resolve/main/databricks-dolly-15k.jsonl \
    -O datasets/databricks-dolly-15k.jsonl


--2024-06-24 00:46:24--  https://huggingface.co/datasets/databricks/databricks-dolly-15k/resolve/main/databricks-dolly-15k.jsonl
Resolving huggingface.co (huggingface.co)... 13.225.131.94, 13.225.131.93, 13.225.131.35, ...
Connecting to huggingface.co (huggingface.co)|13.225.131.94|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/34/ac/34ac588cc580830664f592597bb6d19d61639eca33dc2d6bb0b6d833f7bfd552/2df9083338b4abd6bceb5635764dab5d833b393b55759dffb0959b6fcbf794ec?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27databricks-dolly-15k.jsonl%3B+filename%3D%22databricks-dolly-15k.jsonl%22%3B&Expires=1719449184&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcxOTQ0OTE4NH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy8zNC9hYy8zNGFjNTg4Y2M1ODA4MzA2NjRmNTkyNTk3YmI2ZDE5ZDYxNjM5ZWNhMzNkYzJkNmJiMGI2ZDgzM2Y3YmZkNTUyLzJkZjkwODMzMzhiNGFiZDZiY2ViNTYzNTc2NGRhYjVk

## Pre-process dataset

In [4]:
import json
from pathlib import Path

import pandas as pd
from tqdm.notebook import tqdm

num_samples = 1000

dataset_path = Path().parent / "datasets" / "databricks-dolly-15k.jsonl"
data = pd.read_json(dataset_path, lines=True)
print(data.shape)

prompt_template = "Instruction:\n{instruction}\n\nResponse:\n{response}"

preprocessed_data = []
for _, row in tqdm(data.iterrows()):
    preprocessed_data.append(
        prompt_template.format(
            instruction=row["instruction"],
            response=row["response"],
        )
    )

# Only use a limited number of training examples
preprocessed_data = preprocessed_data[:num_samples]


(15011, 4)


0it [00:00, ?it/s]

## Fine-tune

In [5]:
import keras
import keras_nlp

batch_size = 1

model = keras_nlp.models.Llama3CausalLM.from_preset(str(model_path))
model.summary()

model.backbone.enable_lora(rank=4)
model.summary()

model.preprocessor.sequence_length = 512
optimizer = keras.optimizers.AdamW(
    learning_rate=5e-5,
    weight_decay=0.01,
)
optimizer.exclude_from_weight_decay(var_names=["bias", "scale"])

model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=optimizer,
    weighted_metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

model.fit(preprocessed_data, epochs=1, batch_size=batch_size, verbose=1)


2024-06-24 00:46:26.391217: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-24 00:46:26.424725: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 AVX512_FP16 AVX_VNNI AMX_TILE AMX_INT8 AMX_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-24 00:46:29.194528: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1928] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 79192 MB memory:  -> device: 0, name: CUDA GPU, pci bus id: 0000:43:00.0, compute capability: 9.0
2024-06-24 00:46:29.196389: I tensorflow/core/common_runtime/gpu/g

Preprocessor: "llama3_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Tokenizer (type)                                   ┃                                             Vocab # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ llama3_tokenizer (Llama3Tokenizer)                 │                                             128,256 │
└────────────────────────────────────────────────────┴─────────────────────────────────────────────────────┘

Model: "llama3_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ llama_backbone                │ (None, None, 4096)        │   8,030,261,248 │ padding_mask[0][0],        │
│ (Llama3Backbone)              │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 128256)      │   1,050,673,152 │ llama_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

...

 Non-trainable params: 8,030,261,248 (29.92 GB)

2024-06-24 00:47:11.822947: E tensorflow/core/util/util.cc:131] oneDNN supports DT_INT64 only on platforms with AVX-512. Falling back to the default Eigen-based implementation if present.
I0000 00:00:1719190057.324048   64994 service.cc:145] XLA service 0x7fc2740c9c30 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1719190057.324080   64994 service.cc:153]   StreamExecutor device (0): CUDA GPU, Compute Capability 9.0
I0000 00:00:1719190057.324082   64994 service.cc:153]   StreamExecutor device (1): CUDA GPU, Compute Capability 9.0
2024-06-24 00:47:38.671381: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
W0000 00:00:1719190060.410455   64994 assert_op.cc:38] Ignoring Assert operator compile_loss/sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/assert_equal_1/Assert/Assert
2024-06-24 00:47:41.899200: I extern

   1/1000 ━━━━━━━━━━━━━━━━━━━━ 14:18:08 52s/step - loss: 0.3467 - sparse_categorical_accuracy: 0.3421

I0000 00:00:1719190082.178672   64994 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1000/1000 ━━━━━━━━━━━━━━━━━━━━ 153s 102ms/step - loss: 0.4920 - sparse_categorical_accuracy: 0.5289


## Evaluate

In [6]:
prompts = [
    prompt_template.format(
        instruction="What should I do on a trip to Europe?",
        response="",
    ),
    prompt_template.format(
        instruction="Explain the process of photosynthesis in a way that a child could understand.",
        response="",
    ),
]

sampler = keras_nlp.samplers.TopKSampler(k=5, seed=2)
model.compile(sampler=sampler)

for prompt in prompts:
    print(model.generate(prompt, max_length=256))


I0000 00:00:1719190196.230213   65941 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_732', 4 bytes spill stores, 4 bytes spill loads

I0000 00:00:1719190196.857451   65941 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_291', 192 bytes spill stores, 192 bytes spill loads

I0000 00:00:1719190196.884180   65934 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_291', 468 bytes spill stores, 416 bytes spill loads

I0000 00:00:1719190197.369031   65937 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_357', 4 bytes spill stores, 4 bytes spill loads

I0000 00:00:1719190197.390453   65939 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_357', 192 bytes spill stores, 192 bytes spill loads

I0000 00:00:1719190197.690251   6

Instruction:
What should I do on a trip to Europe?

Response:
Europe has a lot to offer.  First, you should learn to navigate the public transportation system.  You can also learn a few basic phrases in the local language.  If you are visiting Italy, you might consider trying pasta for the first time.  If you are going to the UK, you can visit the Tower of London.  If you are going to Germany, you can visit a beer festival.  Finally, you should consider staying in hostels and budget hotels to save money.

Second, it would be a good idea to learn about European customs and traditions.  For example, you should always say "please" and "thank you" when asking someone for something, and you should never eat while walking in the street.  You should also respect the local culture.  For example, you should avoid public displays of affection in some countries.

Third, it would be a good idea to have a budget and to plan your trip in advance.  You should know how much you want to spend each day 